In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier
from collections import Counter

# === Đọc dữ liệu PROMISE từ file CSV ===
df = pd.read_csv("data/ant-1.4.csv")

# === Chia features và labels ===
X = df.drop(columns=["name", "bug"])
y = df["bug"]
y = y.apply(lambda x: 1 if x > 0 else 0)
print(f"Số lượng mỗi lớp trước SMOTE: {Counter(y)}")

# === Giảm chiều dữ liệu mô phỏng contrastive learning (dùng PCA) ===
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

# === Áp dụng SMOTE-Tomek (fix lỗi thiếu mẫu bằng k_neighbors=1) ===
smote_tomek = SMOTETomek(smote=SMOTE(k_neighbors=1), random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(X_pca, y)

print(f"Số lượng mỗi lớp sau SMOTE-Tomek: {Counter(y_resampled)}")

# === Chia tập train/test ===
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
)

# === Huấn luyện mô hình CatBoost ===
model = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1, verbose=0)
model.fit(X_train, y_train)

#=== Thay doi tap test ===
X_test = pd.read_csv("data/ant-1.5.csv")
y_test = X_test['bug']

X_test = X_test.drop(columns=['name', 'bug'], axis=1)
y_test = y_test.apply(lambda x: 1 if x > 0 else 0)



# === Dự đoán ===
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# === Đánh giá mô hình ===
f1 = f1_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_pred_proba)
acc = accuracy_score(y_test, y_pred)

# === Tính G-Mean ===
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn) if (tp + fn) else 0
specificity = tn / (tn + fp) if (tn + fp) else 0
gmean = np.sqrt(sensitivity * specificity)

# === In kết quả ===
print(f"\n==== Kết quả đánh giá ====")
print(f"F1 Score     : {f1:.4f}")
print(f"ROC AUC      : {roc:.4f}")
print(f"Accuracy     : {acc:.4f}")
print(f"G-Mean       : {gmean:.4f}")


Số lượng mỗi lớp trước SMOTE: Counter({0: 138, 1: 40})
Số lượng mỗi lớp sau SMOTE-Tomek: Counter({0: 136, 1: 136})

==== Kết quả đánh giá ====
F1 Score     : 0.2447
ROC AUC      : 0.7105
Accuracy     : 0.3891
G-Mean       : 0.5433


In [14]:
# F1 Score
f1 = f1_score(y_test, y_pred)

# ROC AUC
roc = roc_auc_score(y_test, y_pred_proba)

# Accuracy
acc = accuracy_score(y_test, y_pred)

# G-Mean
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
gmean = (sensitivity * specificity) ** 0.5

# Kết quả
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f"G-Mean: {gmean:.4f}")


F1 Score: 0.2447
ROC AUC: 0.7105
Accuracy: 0.3891
G-Mean: 0.5433
